In [ ]:
-- This is your Cortex Project.
----------------------------------------------------------
-- SETUP
-----------------------------------------------------------
use role ACCOUNTADMIN;
use warehouse COMPUTE_WH;
use database EPRODUCT;
use schema PUBLIC;

In [ ]:
-- Inspect the first 10 rows of your training data.
-- This is the data we'll use to create your model.
-- select * from ELECTRIC_PRODUCTION limit 10;

-- Prepare your training data. Timestamp_ntz is a required format.
-- Also, only include select columns.
    
CREATE VIEW ELECTRIC_PRODUCTION_v1 AS SELECT
    to_timestamp_ntz(DATE) as DATE_v1,
    IPG2211A2N
FROM ELECTRIC_PRODUCTION;

In [ ]:
select * from ELECTRIC_PRODUCTION limit 10;

In [ ]:
SELECT * FROM ELECTRIC_PRODUCTION_v1;

- CREATE PREDICTIONS
- Create your model.

In [ ]:
CREATE SNOWFLAKE.ML.FORECAST eProduct_forecast_model(
    INPUT_DATA => SYSTEM$REFERENCE('VIEW', 'ELECTRIC_PRODUCTION_v1'),
    TIMESTAMP_COLNAME => 'DATE_v1',
    TARGET_COLNAME => 'IPG2211A2N'
);

In [ ]:
from snowflake.ml.registry import Registry
reg = Registry(session=sp_session, database_name="ML", schema_name="REGISTRY")

In [ ]:
call eProduct_forecast_model!FORECAST(FORECASTING_PERIODS => 14);

In [ ]:
SHOW SNOWFLAKE.ML.FORECAST INSTANCES

In [ ]:
CREATE TABLE my_forecasts_eProduct AS
SELECT * FROM TABLE(EPRODUCT_FORECAST_MODEL!FORECAST(FORECASTING_PERIODS => 14, CONFIG_OBJECT => {'prediction_interval': 0.95}));

In [ ]:
BEGIN
    call eProduct_forecast_model!FORECAST(FORECASTING_PERIODS => 14);
    LET x := SQLID
    CREATE TABLE My_forecasts_2024_11_04 AS SELECT * FROM TABLE(RESULT_SCAN(:x));
END;

In [ ]:
-- View your predictions.
SELECT * FROM my_forecasts_eProduct;

In [ ]:
DESC TABLE my_forecasts_eProduct;

In [ ]:
# Import python packages, pandas for easy data handling
import pandas as pd

# We can also use Snowpark for our analyses!
# With session we can use SQL query data immediately (see 2 cells below)
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
-- Union your predictions with your historical data, then view the results in a chart.
SELECT DATE, IPG2211A2N AS actual, NULL AS forecast, NULL AS lower_bound, NULL AS upper_bound
    FROM ELECTRIC_PRODUCTION
UNION ALL
SELECT ts as DATE, NULL AS actual, forecast, lower_bound, upper_bound
    FROM my_forecasts_eProduct;

In [ ]:
# Easiest way to read data from SQL query results using 
# session data from above. Converting to a pandas dataframe 
# type variable works only if:
# 1) session is loaded (see get_session() command 2 cells above!)
# 2) a table is formed in a previous step (cell) and cellNo is 
# properly defined below:
my_forecast_df = cell14.to_pandas()
my_forecast_df

In [ ]:
#checking the 'DATE' column's data format.
# It should be Datetime or Timestamp for proper time based plotting
type(my_forecast_df['DATE'][0])

In [ ]:
#a way to convert data to Datetime format (if needed)
datetime_df = pd.to_datetime(my_forecast_df['DATE']).apply(lambda x: x.date())
datetime_df

In [ ]:
# subsets of the full dataframe dataset may be selected
plot_df = my_forecast_df[['DATE', 'ACTUAL']]
plot_df

In [ ]:
# chosing the 'ACTUAL' column values and
# only a few from the beginning (head)
my_forecast_df['ACTUAL'].head()

In [ ]:
#import Altair python module for plotting
# (in-built in SnowPark notebook, ...
# other visualization module(s) should be installed first, see docs)
import altair as alt

In [ ]:
alt.Chart(plot_df).mark_circle().encode(
    x= "DATE",
    y = "ACTUAL")

In [ ]:
base = alt.Chart(my_forecast_df).encode(x='DATE')

alt.layer(
    base.mark_line(color='gray').encode(y='ACTUAL'),
    base.mark_line(color='red').encode(y='UPPER_BOUND'),
    base.mark_line(color='blue').encode(y='LOWER_BOUND')
)

In [ ]:
#my_forecast_df[['DATE', 'ACTUAL']]
base = alt.Chart(my_forecast_df.tail(30)).encode(x='DATE').properties(
    title='Electronic Product - 14 days forecast'
)

alt.layer(
    base.mark_line(color='gray').encode(y='ACTUAL'),
    base.mark_line(color='red').encode(y='UPPER_BOUND'),
    base.mark_line(color='blue').encode(y='LOWER_BOUND')
)

As a final step
# INSPECT RESULTS

- Inspect the accuracy metrics of your model.
- Inspect the relative importance of your features, including auto-generated features. 

In [ ]:
-- Inspect the accuracy metrics of your model.
CALL eProduct_forecast_model!SHOW_EVALUATION_METRICS();

In [ ]:
-- Inspect the relative importance of your features, including auto-generated features. 
CALL eProduct_forecast_model!EXPLAIN_FEATURE_IMPORTANCE();